1.1 Fetching and cleaning data

As already mentioned, data preparation tasks will be done with Pandas. Most of the plotting will be handled by the Plotly graph_objects module. Additionaly for one plot we will use the ipywidgets library, which provides widgets for interacting with plots in Jupyter Notebook and JupyterLab. For the time series data we are going to use historic stock market data from Yahoo!Finance API. There is a great Python library called yfinance which we will use to interface with the API.

In [5]:
import pandas as pd
from ipywidgets import widgets as wg
import yfinance as yf
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot






In [9]:
tickers = ["FTSEMIB.MI", "^IXIC"]

df = yf.download(
        tickers=tickers,
        period="3y",
        interval="1wk",
        group_by="column")
        
df.head()

[*********************100%***********************]  2 of 2 completed


Adj Close                   Close                    High  \
           FTSEMIB.MI        ^IXIC FTSEMIB.MI        ^IXIC FTSEMIB.MI   
Date                                                                    
2019-02-11    20212.0  7472.410156    20212.0  7472.410156    20271.0   
2019-02-18    20263.0  7527.540039    20263.0  7527.540039    20361.0   
2019-02-25    20695.0  7595.350098    20695.0  7595.350098    20836.0   
2019-03-04    20484.0  7408.140137    20484.0  7408.140137    21010.0   
2019-03-11    21045.0  7688.529785    21045.0  7688.529785    21112.0   

                               Low                    Open               \
                  ^IXIC FTSEMIB.MI        ^IXIC FTSEMIB.MI        ^IXIC   
Date                                                                      
2019-02-11  7477.279785    19425.0  7290.029785    19435.0  7327.370117   
2019-02-18  7527.540039    20125.0  7430.890137    20232.0  7450.750000   
2019-02-25  7603.040039    20337.0  7485.390137    20428.0  7585.299805   
2019-03-04  7643.660156    20386.0  7332.919922    20820.0  7636.620117   
2019-03-11  7714.959961    20472.0  7442.399902    20588.0  7442.560059   

                Volume               
            FTSEMIB.MI        ^IXIC  
Date                                 
2019-02-11  2379447100  10477050000  
2019-02-18  2188889100   8839310000  
2019-02-25  2391807100  12154480000  
2019-03-04  2506963800  11641280000  
2019-03-11  2515573000  12378400000

In [11]:
df = df.filter(regex="Adj Close")
df = df.dropna()
df.head()

Adj Close             
           FTSEMIB.MI        ^IXIC
Date                              
2019-02-11    20212.0  7472.410156
2019-02-18    20263.0  7527.540039
2019-02-25    20695.0  7595.350098
2019-03-04    20484.0  7408.140137
2019-03-11    21045.0  7688.529785

In [12]:
# use the first date as index 
reference_value = df.iloc[0]
# dividing by the series divides each column by the corresponding element in the series
tmp_df = df.div(reference_value) * 100 - 100
prepared_df = df.copy()

In [13]:
def create_indexed_columns(date, df, top_level_name=""):
    """Returns indexed columns for given dataframe"""

    # find index of the date that is closest to our reference date
    closest_date_index = df.index.get_loc(date, method="nearest")

    # get the values in the initial columns for the reference date 
    reference_values = df.iloc[closest_date_index]['Adj Close']

    # divide initial columns by values at ref. date and store in intermediate df
    inter_df = df['Adj Close'].div(reference_values)*100 - 100

    # create a multindex for the intermediate df using the date as top-level index
    closest_date = df.index[closest_date_index]
    inter_df.columns = pd.MultiIndex.from_product(
                    [[top_level_name if top_level_name else str(closest_date)], inter_df.columns])
    
    return inter_df, closest_date

In [17]:
# instantiate the figure object
fig = go.Figure()

# add a scatter trace for every column
for col in df.columns:
    fig.add_scatter(x=df.index, y=df[col], name=col[1])

# change the scale to logarithmic and add title
fig.update_layout(
    yaxis=dict(type="log"),
    title=f"Index prices for {df.index[0].strftime('%B %Y')} - " \
          +                f"{df.index[-1].strftime('%B %Y')}"
)

fig.show()